In [5]:
import settings
import helpers
import glob
import os
import cv2
import scipy.misc
import dicom
import numpy
from multiprocessing import Pool

In [3]:
def cv_flip(img,cols,rows,degree):
    M = cv2.getRotationMatrix2D((cols/2,rows/2),degree,1.0)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst

In [7]:
def extract_dicom_images(clean_targetdir_first = False, only_patient_id=None):
    print("Extracting Images")
    
    target_dir = settings.TRAIN_EXTRACTED_DIR
    if clean_targetdir_first and only_patient_id is not None:
        print("Cleaning Target Directory")
        for file_path in glob.glob(target_dir + ".*"):
            os.remove(file_path)
    
    if only_patient_id is None:
        dirs = os.listdir(settings.TRAIN_DIR_RAW)
        if True:
            pool = Pool(settings.WORKER_POOL_SIZE)
            pool.map(extract_dicom_images_patient,dirs)
        else:
            for dir_entry in dirs:
                extract_dicom_images_patient(dir_entry)
    else:
        extract_dicom_images_patient(only_patient_id)

In [9]:
def extract_dicom_images_patient(src_dir):
    target_dir = settings.TRAIN_EXTRACTED_DIR
    print("Dir: ", src_dir)
    dir_path = settings.TRAIN_DIR_RAW + src_dir + "/" + settings.BEFORE_DIR
    patient_id = src_dir
    slices = helpers.load_scan(dir_path)
    print(len(slices),"\t",slices[0].SliceThickness,"\t",slices[0].PixelSpacing)
    print("Orientation: ", slices[0].ImageOrientationPatient)
    cos_value = (slices[0].ImageOrientationPatient[0])
    cos_degree = round(math.degrees(math.acos(cos_value)),2)
    
    pixels = helpers.get_pixels_hu(slices)
    image = pixels
    print(image.shape)
    invert_order = slices[1].ImagePositionPatient[2]>slices[0].ImagePositionPatient[2]
    print("Invert order: ", invert_order, " - ", slices[1].ImagePositionPatient[2],",", 
          slices[0].ImagePositionPatient[2])
    
    pixel_spacing = slices[0].PixelSpacing
    pixel_spacing.append(slices[0].SliceThickness)
    image = helpers.rescale_patient_images(image,pixel_spacing,settings.TARGET_VOXEL_MM)
    if not invert_order:
        image = numpy.flipud(image)
    for i in range(image.shape[0]):
        patient_dir = target_dir + patient_id + "/" + settings.BEFORE_DIR
        if not os.path.exists(patient_dir):
            print("couldn't find path to folder...")
        img_path = patient_dir + "img_" + str(i).rjust(4,'0') + "_i.png"
        org_img = image[i]
        if cos_degree > 0.0:
            org_img = cv_flip(org_img,org_img.shape[1],org_img[0],cos_degree)
        img, mask = helpers.get_segmented_lungs(org_img.copy())
        org_img = helpers.normalize_hu(org_img)
        cv2.imwrite(img_path, org_img*255)
        cv2.imwrite(im_path.replace("_i.png","_m.png"), mask*255)
        

In [10]:
extract_dicom_images()

Extracting Images


AttributeError: module 'settings' has no attribute 'TRAIN_EXTRACTED_DIR'